In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
from sklearn import svm
from sklearn import preprocessing
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
# from sklearn.model_selection import StratifiedKFold

In [8]:
proto_map = {'tcp': 1, 'udp': 2, 'rtp': 3, 'rtcp': 4}
direction_map = {'   ->': 1, '  <?>': 2, '   ?>': 3, '  <->': 4}
state_map = {
    'S_': 1,
    'FSPA_FSPA': 2,
    'SRPA_SA': 3,
    'SRPA_SPA': 4,
    'FSPA_FSRPA': 5,
    'S_RA': 6,
    'FSPA_SRPA': 7,
    'FSRPA_SPA': 8,
    'FSRPA_SA': 9,
    'PA_R': 10,
    'SRPA_FSPA': 11,
    'SPA_FSRPA': 12,
    'FSRPA_FSPA': 13,
    'FSPA_FSA':  14,
    'CON': 15,
    'FPA_R': 16,
    'SRPA_FSRPA': 17,
    'RST': 18,
    'SPA_SRPA': 19,
    'SR_SA': 20,
    'INT': 21,
    'SA_R': 22,
    'SPA_SPA': 23,
    'SR_A': 24,
    'FIN': 25,
    'FPA_':  26,
    'SPA_SA': 27,
    'REQ': 28,
    'RPA_R': 29,
    'FA_R': 30,
    'FSPA_SA': 40,
    'FSRPA_SRA': 41,
    'A_R': 42,
    'FSRPA_FSRPA': 43,
    'SRA_R': 44,
    'SPA_FSPA': 45,
    'SRPA_R': 46,
    'SPA_SRA': 47,
    'RA_R': 48,
    'RA_': 49,
    'SRA_SRA': 50,
    'S_SA': 51,
    'R_': 52,
    'A_RPA': 53,
    'FRPA_': 54,
    'A_': 55,
    'SA_SA': 56
}

In [40]:
def get_predict(X_train, X_test):
    clf = svm.OneClassSVM(kernel="rbf", gamma='scale', cache_size=3000)
    scaler = preprocessing.StandardScaler().fit(X_train)
#     scaler = preprocessing.RobustScaler().fit(X_train)
    clf.fit(scaler.transform(X_train))
    return clf.predict(scaler.transform(X_test))

In [9]:
feature_df = df.copy()
feature_df = feature_df.loc[feature_df['Label'] == 1].reset_index(drop=True)
feature_df['Proto'] = feature_df['Proto'].map(proto_map)
feature_df['Dir'] = feature_df['Dir'].map(direction_map)
feature_df['State'] = feature_df['State'].map(state_map)
feature_df = feature_df.drop(columns=['StartTime', 'Label', 'SrcAddr', 'Sport', 'DstAddr', 'Dport', 'sTos', 'dTos', 'is_fwd' ])

In [34]:
X = feature_df.iloc[:80000].to_numpy()
kf = KFold(n_splits=3)
results = []
score = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    start = time.time()
    result = get_predict(X_train, X_test)
    end = time.time()
    results.append(result)
    print('One Done')
    print('Time')
    print(end - start)
    print('Percentage')
    total = len(result)
    count = 0
    for item in result:
        if item == 1:
            count = count + 1
    print(count/total * 100)
    score.append(count/total * 100)
mean = (score[0] + score[1] + score[2])/3
mean

In [ ]:
# results = []
# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     results.append(get_predict(X_train, X_test))
#     print('One Done')

In [ ]:
# enc = preprocessing.OneHotEncoder()
# X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
# enc.transform([['female', 'from US', 'uses Safari'],
#                ['male', 'from Europe', 'uses Safari']]).toarray()
#Impute mean
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp.fit(test_2['Time2FlowFwd_10T'].to_numpy())
# kf = KFold(n_splits=3, shuffle=True)
# kf = StratifiedKFold(n_splits=3, shuffle=True) 